In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# Семинар 5 

Сегодня мы будем использовать данные, с этого [соревнования](https://www.kaggle.com/c/bigdata-team-taxi-trip-distance/overview). Для начала, скачаем данные.

In [2]:
train = pd.read_csv('./data/train.csv', index_col='id')
test = pd.read_csv('./data/test.csv', index_col='id')

FileNotFoundError: ignored

Взглянем на данные:

In [ ]:
train.head(3)

In [ ]:
test.head(3)

Посмотрим на столбцы, которые заданы не числами: 

In [ ]:
train.dtypes[train.dtypes=='object']

Посмотрим на них внимательно и сразу пофиксим: 

In [ ]:
train['store_and_fwd_flag'].value_counts()

In [ ]:
test['store_and_fwd_flag'].value_counts()

Просто заменим на `{0,1}`

In [ ]:
train['store_and_fwd_flag'] = train['store_and_fwd_flag'].map({'N':0, 'Y':1})
test['store_and_fwd_flag'] = test['store_and_fwd_flag'].map({'N':0, 'Y':1})

Посмотрим, на два других столбца:

In [ ]:
train[['tpep_dropoff_datetime','tpep_pickup_datetime']].head()

Напишем функцию, которая будет доставать максимум полезных фичей:

In [ ]:
def date_to_cat(df, date):
    df['day'] = pd.to_datetime(df[date]).dt.day
    df['month'] = pd.to_datetime(df[date]).dt.month
    df['year'] = pd.to_datetime(df[date]).dt.year
    df['hour'] = pd.to_datetime(df[date]).dt.hour
    df['minute'] = pd.to_datetime(df[date]).dt.minute
    df['dayofweek'] = pd.to_datetime(df[date]).dt.dayofweek
    df.drop(date, inplace=True, axis=1)


In [ ]:
day_columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
for i in day_columns: 
    date_to_cat(train, i)
    date_to_cat(test, i)

Достанем из обучающей выборки целевую переменную:

In [ ]:
y = train['trip_distance']
train = train.drop('trip_distance', axis=1)

In [ ]:
test.shape, train.shape

## Пришло время учить модели

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
                    train, y, train_size=0.7, test_size=0.3)

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

In [ ]:
mean_squared_error(y_val, rf.predict(x_val))

In [ ]:
trn_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_val, label=y_val)

In [ ]:
param = {
    'boost': 'gbdt',
    'metric':'rmse',
    'num_threads': 12,
    'objective': 'regression', 
    'learning_rate': 0.2, # Установим скорость обучения
    'max_depth': 2, # Ограничим дерево 
}

In [ ]:
history = {} # Будем тут хранить историю
clf = lgb.train(param, trn_data, num_boost_round=5000, valid_sets = [trn_data, val_data], 
                verbose_eval=100,  evals_result=history, early_stopping_rounds = 100)

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(history['training']['rmse'], label='Train')
plt.plot(history['valid_1']['rmse'], label='Test')
plt.xlabel('Iterations')
plt.ylabel('RMSE')
plt.title('История обучения модели')
plt.legend()
plt.show()

## Отправим решение на Kaggle

In [ ]:
solutions = pd.read_csv('./data/sample.csv', index_col='id')

In [ ]:
rf_pred = rf.predict(test)

In [ ]:
lgb_pred = clf.predict(test)

In [ ]:
solutions['trip_distance'] = rf_pred
solutions['trip_distance'].apply(lambda x: 0 if x < 0 else x)
solutions.to_csv('baseline_1-rf.csv')

In [ ]:
solutions['trip_distance'] = lgb_pred
solutions['trip_distance'].apply(lambda x: 0 if x < 0 else x)
solutions.to_csv('baseline_1-lgbm.csv')

## Минутка не смешного, профессионального юмора:
<img src='random.jpeg', width=300>

## Some ideas

In [ ]:
plt.figure(figsize=(20, 4))
sns.kdeplot(y.values)
plt.show()

Целевая переменная распредлена не нормально и она строго больше 0

Давайте прологарифмируем

In [ ]:
plt.figure(figsize=(20, 4))
sns.kdeplot(np.log(y.values))
plt.show()

## О соревновании

- Deadline: 29 июня в 17:00 (Алма-Аты)
- Не забудьте, выбрать 2 финальных решения
- Победители (топ 3) - Показывают __воспроизводимый__ код и рассказывают о своем решении 
- Нужно написать свое имя в поле 'Team'
- 10 попыток в день! (обновление в 6 утра)
- Тест разделен на две части: Публичный и приватный (только после дедлайна) лидерборд

<img src='shakeup.png', width=500>

# Feedback
https://rebrand.ly/ml_yessenov_2019_feedback_05